In [4]:
import pandas as pd
import requests
import json
from tqdm import tqdm

# Cargar el DataFrame
df = pd.read_csv('Datasets/results/processed_after_ner_labels.csv', encoding='utf-8', sep=';')

# Asegurarse de que los valores de 'Filtered_NER' estén en formato de listas
df['Filtered_NER'] = df['Filtered_NER'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Aplanar las listas y obtener los términos únicos
all_terms = [item for sublist in df['Filtered_NER'] for item in sublist]

# Calcular la frecuencia de cada término
term_frequencies = pd.Series(all_terms).value_counts()

# Filtrar los términos más frecuentes (por ejemplo, los 10 más comunes)
most_frequent_terms = term_frequencies.head(10).index  # Puedes ajustar el número si necesitas más o menos

print("Most frequent terms:", most_frequent_terms)

def get_wikidata_labels(term):
    """
    Busca el término en Wikidata y devuelve las etiquetas en inglés y español.
    """
    url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={term}&language=en&limit=1&format=json"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Levanta un error si la respuesta es incorrecta
        data = response.json()

        # Verificar si existen resultados
        if 'search' in data and len(data['search']) > 0:
            # Obtener el ID del primer resultado
            wikidata_id = data['search'][0]['id']
            # Ahora obtener las etiquetas usando el ID
            labels_url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={wikidata_id}&props=labels&languages=en|es&format=json"
            labels_response = requests.get(labels_url, timeout=10)
            labels_response.raise_for_status()
            labels_data = labels_response.json()
            
            # Verificar y extraer las etiquetas en inglés y español
            if 'entities' in labels_data and wikidata_id in labels_data['entities']:
                labels = labels_data['entities'][wikidata_id].get('labels', {})
                return {lang: label['value'] for lang, label in labels.items() if lang in ['en', 'es']}
            else:
                return {}
        else:
            return {}  # Retorna un diccionario vacío si no hay resultados
    except requests.exceptions.RequestException as e:
        print(f"Error buscando el término '{term}': {e}")
        return {}

# Crear un diccionario para almacenar los resultados
wikidata_dict = {}

# Iterar sobre los términos más frecuentes y mostrar progreso con tqdm
for term in tqdm(most_frequent_terms, desc="Fetching Wikidata labels", unit="term"):
    labels = get_wikidata_labels(term)
    if labels:  # Si se encontraron etiquetas, agregarlas al diccionario
        wikidata_dict[term] = labels
    else:
        print(f"No se encontraron etiquetas para el término: {term}")

# Guardar el diccionario resultante en un archivo JSON
with open('wikidata_labels_most_frequent.json', 'w', encoding='utf-8') as f:
    json.dump(wikidata_dict, f, ensure_ascii=False, indent=4)

print("Diccionario guardado como 'wikidata_labels_most_frequent.json'.")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Daniel\anaconda3\envs\IA\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Daniel\anaconda3\envs\IA\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\Daniel\anaconda3\envs\IA\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\Daniel\anaconda3\envs\IA\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [3]:
get_wikidata_labels('cilantrillo')

NameError: name 'get_wikidata_labels' is not defined